## Project 4 
#### NoteBook 1 - Scraping
## Submitted by Roshan Lulu

## Business Case

You're working as a data scientist for a contracting firm that's rapidly expanding. Now that they have their most valuable employee (you!), they need to leverage data to win more contracts. Your firm offers technology and scientific solutions and wants to be competitive in the hiring market. Your principal has two main objectives:
1. Determine the industry factors that are most important in predicting the salary amounts for these data.
2. Determine the factors that distinguish job categories and titles from each other. For example, can required skills accurately predict job title?

To limit the scope your principal has suggested that you focus on data-related job postings, e.g. data scientist, data analyst, research scientist, business intelligence, etc. You may also want to decrease the scope by selecting a single region.

Aggregators like [Indeed.com](https://www.indeed.com) regularly pool job postings from a variety of markets and industries. You will be scraping data from Indeed.com or other job aggregators to answer these two questions.

---

### Step 1 - Get the Job Posting data from the website

- Identify the Job titles to scrap
- Identify the element/child relationships and get the attributes
- Baby step - Get 1 job posting scraped


### A. IMPORT LIBRARIES

In [1]:
# import libraries
import scrapy
from scrapy.http import Request
from scrapy import Selector
import requests
import pandas as pd


### B. PARSE SEARCH RESULTS FOR THE JOB POST LINKS

In [2]:
# # define the search urls
# urls = [{'page':1,'type': 'data-analyst', 'url':'https://www.seek.com.au/Data-Analyst-jobs/in-All-Sydney-NSW'},
#                 {'page':1,'type': 'data-science', 'url':'https://www.seek.com.au/Data-Science-jobs/in-All-Sydney-NSW'},
#                  {'page':1, 'type': 'business-intelligence', 'url':'https://www.seek.com.au/Business-Intelligence-jobs/in-All-Sydney-NSW'}]

job_desc_i_want = ['Data-Science', 'Data-Analyst', 'Business-Intelligence', 'Research-Scientist', 'Business-Analyst']
where_is_the_job = ['-jobs/in-All-Sydney-NSW', '-jobs/in-All-Melbourne-VIC', '-jobs/in-All-Brisbane-QLD']
search_urls = []

for index in range(0, len(job_desc_i_want)):
    for item in range(0, len(where_is_the_job)):
        job_dict = {}
        job_dict['loc'] = where_is_the_job[item].split('/')[1]
        job_dict['type'] = job_desc_i_want[index]
        job_dict['page'] = 1
        job_dict['url'] = 'https://www.seek.com.au/' + job_desc_i_want[index] + where_is_the_job[item]
        search_urls.append(job_dict)

search_urls

[{'loc': 'in-All-Sydney-NSW',
  'page': 1,
  'type': 'Data-Science',
  'url': 'https://www.seek.com.au/Data-Science-jobs/in-All-Sydney-NSW'},
 {'loc': 'in-All-Melbourne-VIC',
  'page': 1,
  'type': 'Data-Science',
  'url': 'https://www.seek.com.au/Data-Science-jobs/in-All-Melbourne-VIC'},
 {'loc': 'in-All-Brisbane-QLD',
  'page': 1,
  'type': 'Data-Science',
  'url': 'https://www.seek.com.au/Data-Science-jobs/in-All-Brisbane-QLD'},
 {'loc': 'in-All-Sydney-NSW',
  'page': 1,
  'type': 'Data-Analyst',
  'url': 'https://www.seek.com.au/Data-Analyst-jobs/in-All-Sydney-NSW'},
 {'loc': 'in-All-Melbourne-VIC',
  'page': 1,
  'type': 'Data-Analyst',
  'url': 'https://www.seek.com.au/Data-Analyst-jobs/in-All-Melbourne-VIC'},
 {'loc': 'in-All-Brisbane-QLD',
  'page': 1,
  'type': 'Data-Analyst',
  'url': 'https://www.seek.com.au/Data-Analyst-jobs/in-All-Brisbane-QLD'},
 {'loc': 'in-All-Sydney-NSW',
  'page': 1,
  'type': 'Business-Intelligence',
  'url': 'https://www.seek.com.au/Business-Intelli

In [3]:
pagelinks_df = pd.DataFrame(columns = ('Job_Desc', 'Job_Loc','Page_Num', 'url_link'))

In [4]:
# Define the start request function
def get_all_page_links_per_job(url, job_desc, job_loc, page_num=1):
    # Get the html response from the first page url
    response = requests.get(url)
    # get the link to the next page
    next_page_anchor  =  Selector(text=response.text).xpath("//a[@data-automation='page-next']/@href").extract()
    # check if there is a next page
    if len(next_page_anchor) > 0:
        # Save the current url details before finding the next one
        pagelinks_df.loc[len(pagelinks_df),:] = [job_desc, job_loc, page_num, url]
        # Start searching for next page link
        # Recursively call to get the links in next anchor page
        page_num += 1
        next_page_anchor = "https://www.seek.com.au" + next_page_anchor[0]
        get_all_page_links_per_job(next_page_anchor, job_desc, job_loc, page_num)
    else:
        pass

In [5]:
def parse_all_first_pages(urls=search_urls):
    for url in urls:
        get_all_page_links_per_job(url['url'], url['type'], url['loc']) 

#Parse all the required job search results and get the links to the job postings
parse_all_first_pages()

In [6]:
# Display the dataframe obtained
pagelinks_df.head()

,Job_Desc,Job_Loc,Page_Num,url_link
0,Data-Science,in-All-Sydney-NSW,1,https://www.seek.com.au/Data-Science-jobs/in-A...
1,Data-Science,in-All-Sydney-NSW,2,https://www.seek.com.au/Data-Science-jobs/in-A...
2,Data-Science,in-All-Sydney-NSW,3,https://www.seek.com.au/Data-Science-jobs/in-A...
3,Data-Science,in-All-Sydney-NSW,4,https://www.seek.com.au/Data-Science-jobs/in-A...
4,Data-Science,in-All-Melbourne-VIC,1,https://www.seek.com.au/Data-Science-jobs/in-A...


In [7]:
# Get all the job posting links from each page
list_of_posts = []
for index in range(0, len(pagelinks_df)):
    # Get the response text from the url
    url = pagelinks_df.iloc[index, 3]
    response = requests.get(url)
    # Parse the response text
    # parse_job_posting(response)
    post = Selector(text=response.text).xpath('//a[@data-automation="jobTitle"]/@href').extract()
    list_of_posts.append(post)

pagelinks_df['Job_Posts'] = list_of_posts
pagelinks_df

,Job_Desc,Job_Loc,Page_Num,url_link,Job_Posts
0,Data-Science,in-All-Sydney-NSW,1,https://www.seek.com.au/Data-Science-jobs/in-A...,[/job/33394134?type=standout&tier=no_tier&pos=...
1,Data-Science,in-All-Sydney-NSW,2,https://www.seek.com.au/Data-Science-jobs/in-A...,[/job/33378604?type=standard&tier=no_tier&pos=...
2,Data-Science,in-All-Sydney-NSW,3,https://www.seek.com.au/Data-Science-jobs/in-A...,[/job/33344571?type=standout&tier=no_tier&pos=...
3,Data-Science,in-All-Sydney-NSW,4,https://www.seek.com.au/Data-Science-jobs/in-A...,[/job/33309863?type=standout&tier=no_tier&pos=...
4,Data-Science,in-All-Melbourne-VIC,1,https://www.seek.com.au/Data-Science-jobs/in-A...,[/job/33302293?type=standout&tier=no_tier&pos=...
5,Data-Science,in-All-Melbourne-VIC,2,https://www.seek.com.au/Data-Science-jobs/in-A...,[/job/33286172?type=standout&tier=no_tier&pos=...
6,Data-Science,in-All-Melbourne-VIC,3,https://www.seek.com.au/Data-Science-jobs/in-A...,[/job/33343221?type=standard&tier=no_tier&pos=...
7,Data-Science,in-All-Melbourne-VIC,4,https://www.seek.com.au/Data-Science-jobs/in-A...,[]
8,Data-Analyst,in-All-Sydney-NSW,1,https://www.seek.com.au/Data-Analyst-jobs/in-A...,[/job/33188669?type=standard&tier=no_tier&pos=...
9,Data-Analyst,in-All-Sydney-NSW,2,https://www.seek.com.au/Data-Analyst-jobs/in-A...,[/job/33359532?type=standard&tier=no_tier&pos=...


### C. PARSE JOB POSTINGS FOR THE TEXT TAGS!

In [8]:
# Go through each page link and get the job post content:
# Description, Job_Title, Salary, Location, Work_Type, Classification,
# DatePosted, Postedby
# Define the start request function
rows = []

In [9]:
for index in range(0, len(pagelinks_df)):
    # Get the response text from the url
    print('\nPage', index)
    urls = pagelinks_df.iloc[index, 4]
    keyword = pagelinks_df.iloc[index, 0]
    pagenum = pagelinks_df.iloc[index, 2]
    count = 0
    print('\n No of urls to parse:', len(urls))
    for url in urls:
        count = count + 1
        print("\nParshing url: ", count, '/', len(urls))
        url = 'https://www.seek.com.au' + url
        response = requests.get(url)
        title = Selector(text=response.text).xpath('//div[@data-automation="joblisting-index"]/div[@class="grid_6"]/h1/text()').extract_first()
        adv = Selector(text=response.text).xpath('//div[@data-automation="joblisting-index"]/div[@class="grid_6"]/span/text()').extract_first()
        jd1 = Selector(text=response.text).xpath('//div[@class="templatetext"]/p//text()').extract()
        jd2 = Selector(text=response.text).xpath('//div[@class="templatetext"]/ul/li//text()').extract()
        jd3 = Selector(text=response.text).xpath('//div[@class="templatetext"]/text()').extract()
        salary = Selector(text=response.text).xpath('//div[@data-automation="joblisting-index"]/div/div[@class="mod-job-details"]/ul/li/div[@itemprop="baseSalary"]/text()').extract_first()
        worktype = Selector(text=response.text).xpath('//div[@data-automation="joblisting-index"]/div/div[@class="mod-job-details"]/ul/li/div[@itemprop="employmentType"]/text()').extract_first()
        classif = Selector(text=response.text).xpath('//div[@data-automation="joblisting-index"]/div/div[@class="mod-job-details"]/ul/li/div/span[@itemprop="industry"]/text()').extract_first()
        region = Selector(text=response.text).xpath('//div[@data-automation="joblisting-index"]/div/div[@class="mod-job-details"]/ul/li/div/span[@itemprop="addressRegion"]//text()').extract_first()
        location = Selector(text=response.text).xpath('//div[@data-automation="joblisting-index"]/div/div[@class="mod-job-details"]/ul/li/div/span[@itemprop="addressLocality"]//text()').extract_first()
        # Append the scraped items to the list of rows
        rows.append([keyword, pagenum, title, region, location, salary, worktype, adv, classif, jd1, jd2, jd3, url])


Page 0

 No of urls to parse: 20

Parshing url:  1 / 20

Parshing url:  2 / 20

Parshing url:  3 / 20

Parshing url:  4 / 20

Parshing url:  5 / 20

Parshing url:  6 / 20

Parshing url:  7 / 20

Parshing url:  8 / 20

Parshing url:  9 / 20

Parshing url:  10 / 20

Parshing url:  11 / 20

Parshing url:  12 / 20

Parshing url:  13 / 20

Parshing url:  14 / 20

Parshing url:  15 / 20

Parshing url:  16 / 20

Parshing url:  17 / 20

Parshing url:  18 / 20

Parshing url:  19 / 20

Parshing url:  20 / 20

Page 1

 No of urls to parse: 20

Parshing url:  1 / 20

Parshing url:  2 / 20

Parshing url:  3 / 20

Parshing url:  4 / 20

Parshing url:  5 / 20

Parshing url:  6 / 20

Parshing url:  7 / 20

Parshing url:  8 / 20

Parshing url:  9 / 20

Parshing url:  10 / 20

Parshing url:  11 / 20

Parshing url:  12 / 20

Parshing url:  13 / 20

Parshing url:  14 / 20

Parshing url:  15 / 20

Parshing url:  16 / 20

Parshing url:  17 / 20

Parshing url:  18 / 20

Parshing url:  19 / 20

Parshing url:

In [10]:
columns = ('keyword', 'pagenum', 'title', 'region', 'location', 'salary', 'worktype', 'adv', 'classif', 'jd1', 'jd2', 'jd3','url')
job_posts_df = pd.DataFrame(rows, columns = columns)

In [11]:
print(len(job_posts_df))
job_posts_df.tail(120)


2831


,keyword,pagenum,title,region,location,salary,worktype,adv,classif,jd1,jd2,jd3,url
2711,Business-Analyst,5,Senior Business Analysts,Northern Suburbs,Brisbane,$93 934 - $100 624 p.a.,Contract/Temp,Metro North Hospital & Health Service -,Information & Communication Technology,"[Remuneration value up to $114 808 p.a., compr...",[],"[\r\n, \r\n, \r\n, \r\n, \r\n, \r\n, \r\n, \r\...",https://www.seek.com.au/job/32777669?type=stan...
2712,Business-Analyst,5,Senior Business Analysts,None,Brisbane,$93 934 - $100 624 p.a.,Contract/Temp,Metro North Hospital & Health Service -,Government & Defence,"[Remuneration value up to $114 808 p.a., compr...",[],"[\r\n, \r\n, \r\n, \r\n, \r\n, \r\n, \r\n, \r\...",https://www.seek.com.au/job/32777671?type=stan...
2713,Business-Analyst,5,Business Analyst - DAF ITP - 6 month contract,None,Brisbane,None,Contract/Temp,Hays Information Technology -,Information & Communication Technology,"[Business Analyst Job in Brisbane CBD, DAF ITP...",[Capture and/or review/update end-to-end curre...,"[\r\n, \r\n, \r\n, \r\n, \r\n, \r\n, \r\n\r\n ...",https://www.seek.com.au/job/33230729?type=stan...
2714,Business-Analyst,5,Business Analyst,None,Brisbane,None,Contract/Temp,Talent – Winner ‘Seek Large Recruitment Agency...,Information & Communication Technology,[],[Capture and/or review/update end-to-end curre...,[\r\nYou have the exciting opportunity to join...,https://www.seek.com.au/job/33233006?type=stan...
2715,Business-Analyst,5,Business Analyst - Process Mapping,None,Brisbane,None,Contract/Temp,Quality People -,Information & Communication Technology,[Our client URGENTLY requires a Business Analy...,[Extensive experience in Process Analysis usin...,"[\r\n \r\n, \r\n, \r\n, \r\n, \r\n, \...",https://www.seek.com.au/job/33228604?type=stan...
2716,Business-Analyst,5,Senior Business Analyst/Financial Services,CBD & Inner Suburbs,Brisbane,None,Contract/Temp,M&T Resources -,Information & Communication Technology,[M&T Resources is an equal opportunities recru...,[],"[On behalf of our client, a market leading org...",https://www.seek.com.au/job/33226005?type=stan...
2717,Business-Analyst,5,Business Analyst,None,Brisbane,None,Contract/Temp,Paxus Australia Pty Ltd -,Information & Communication Technology,[],[Analysis of user requirements to identify the...,[Our major Qld Government client is seeking an...,https://www.seek.com.au/job/33225828?type=stan...
2718,Business-Analyst,5,Business Analyst (Commercial Enterprise Team),CBD & Inner Suburbs,Brisbane,None,Full Time,Best Practice Software -,Information & Communication Technology,"[This is Best Practice., As one of Australasia...",[Experience in business analysis/consulting en...,"[\r\n, \r\n, \r\n, \r\n, \r\n, \r\n, \r\n, \r\...",https://www.seek.com.au/job/33224536?type=stan...
2719,Business-Analyst,5,Senior Business Analyst (Commercial Enterprise...,CBD & Inner Suburbs,Brisbane,None,Full Time,Best Practice Software -,Information & Communication Technology,"[This is Best Practice., As one of Australasia...",[Experience in leading business analysis/consu...,"[\r\n, \r\n, \r\n, \r\n, \r\n, \r\n, \r\n, \r\...",https://www.seek.com.au/job/33220938?type=stan...
2720,Business-Analyst,5,Business Analyst,CBD & Inner Suburbs,Brisbane,None,Full Time,Compare the Market -,Marketing & Communications,[],[Develop User Stories and Business Requirement...,"[\r\n, \r\nLaunched in 2012, Compare the Marke...",https://www.seek.com.au/job/33224990?type=stan...


### D. Create SQL DB

In [12]:
# Write save it to a csv file
job_posts_df.to_csv('.database/Job_Posts_final.csv', index = False)

pagelinks_df.to_csv('.database/Search_Results_final.csv', index = False)

In [13]:
import sqlite3

posts = pd.read_csv('.database/Job_Posts_final.csv')
results = pd.read_csv('.database/Search_Results_final.csv')
sqlite_db = './jobSeek_final.sqlite'
conn = sqlite3.connect(sqlite_db) 
c = conn.cursor()

In [14]:
posts.to_sql('job_posts',                 # Name of the table
            con=conn,                    # The handle to the file that is setup
            if_exists='replace',         # Overwrite, append, or fail
            index=False)                 # Add index as column

In [15]:
results.to_sql('search_results',         # Name of the table
            con=conn,                    # The handle to the file that is setup
            if_exists='replace',         # Overwrite, append, or fail
            index=False)                 # Add index as column

In [16]:
c.close()